## Bias og maskinlæring - et syntetisk eksempel

Boilerplate

In [ ]:
import fairlearn
import matplotlib.pyplot as plt
import numpy as np
import pandas_profiling as pp
import pandas as pd
import seaborn as sns
import dataframe_image as dfi
from sklearn import metrics as skm

from fairlearn.metrics import MetricFrame, count, selection_rate
from fairlearn.postprocessing import ThresholdOptimizer, plot_threshold_optimizer
from sklearn.inspection import PartialDependenceDisplay
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from fairlearn_demo.synthetic import SyntheticCreditData

sns.set()

## Hente data

In [ ]:
df = SyntheticCreditData().sample(25000)
df.describe()

Styling og print av datautsnitt

In [ ]:
df.loc[:, "inntekt_formatert"] = df["inntekt"].map("{:,d}".format) + " kr"
df.loc[:, "inntekt_formatert"] = df["inntekt_formatert"].map(
    lambda x: x.replace(",", " ")
)
styled = (
    df[["kjonn", "etnisitet", "alder", "utdanning", "inntekt_formatert", "mislighold"]]
    .sample(50)
    .copy()
    .rename(columns={"inntekt_formatert": "inntekt"})
    .sample(12)
    .style.highlight_max(subset=["mislighold"], props="color:red;font-weight:bold")
    .highlight_max(
        subset=["kjonn"], color="rgb(220, 220, 240)", props="font-weight:bold"
    )
    .highlight_max(
        subset=["etnisitet"], color="rgb(200, 220, 240)", props="font-weight:bold"
    )
    .highlight_min(
        subset=["utdanning"], color="rgb(200, 200, 200)", props="font-weight:light"
    )
    .hide(axis="index")
    .set_properties(**{"text-align": "center"})
)
dfi.export(styled, "datasample.png")
dfi.export(
    styled.hide_columns(["kjonn", "etnisitet"]), "datasample_nogender_noetnicity.png"
)

#### Splitter i trening og testsett, og definerer sensitive verdier

In [ ]:
A = df[["kjonn", "etnisitet"]]
y = df["mislighold"]
X = df.drop(labels=["kjonn", "etnisitet", "mislighold", "inntekt_formatert"], axis=1)


X = pd.get_dummies(X, drop_first=True)
sc = StandardScaler()
X_scaled = sc.fit_transform(X)
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)
X_train, X_test, y_train, y_test, A_train, A_test = train_test_split(
    X_scaled, y, A, test_size=0.3, random_state=1234, stratify=y
)

#### Trening av naiv modell

In [ ]:
unmitigated_model = MLPClassifier(
    max_iter=2000,
    hidden_layer_sizes=(50, 50),
    learning_rate="adaptive",
    learning_rate_init=0.0003,
)
unmitigated_model.fit(X_train, y_train)

Undersøker diskrimineringsevne ved roc auc

In [ ]:
fig1 = plt.gcf()
FPR, TPR, _ = skm.roc_curve(y_test, unmitigated_model.predict_proba(X_test)[:, 1])
FPR_all_zeros, TPR_all_zeros, _ = skm.roc_curve(y_test, np.zeros(len(y_test)))
# FPR, TPR, _ = skm.roc_curve(y_test, -X_test["inntekt"])

fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12, 4))
roc_display = skm.RocCurveDisplay(fpr=FPR, tpr=TPR)
roc_display_all_zeros = skm.RocCurveDisplay(fpr=FPR_all_zeros, tpr=TPR_all_zeros)

roc_display.plot(ax[0])
roc_display_all_zeros.plot(ax[1])
fig.savefig("roc.png", dpi=100)

#### Undersøke rettferdighetsmetrikker

In [ ]:
gender_test = A_test[["kjonn"]]
etnicity_test = A_test[["etnisitet"]]
metric_frame = MetricFrame(
    metrics={
        "accuracy": skm.accuracy_score,
        "precision": skm.precision_score,
        "auc": skm.roc_auc_score,
        "recall": skm.recall_score,
        "selection_rate": selection_rate,
        "count": count,
        "TPR": fairlearn.metrics.true_positive_rate,
        "FPR": fairlearn.metrics.false_positive_rate,
    },
    sensitive_features=A_test,
    y_true=y_test,
    y_pred=unmitigated_model.predict(X_test),
)
metric_frame.overall

In [ ]:
# Styler output
to_blog = (
    (
        metric_frame.by_group.drop(
            ["accuracy", "precision", "auc", "recall", "count"], axis=1
        )
        .copy()
        .style.highlight_max(subset=["FPR"], props="color:red;font-weight:bold")
        .highlight_min(subset=["FPR"], props="color:green;font-weight:bold")
        .highlight_max(subset=["selection_rate"], props="color:red;font-weight:bold")
        .highlight_min(subset=["selection_rate"], props="color:green;font-weight:bold")
        .highlight_max(subset=["TPR"], props="color:green;font-weight:bold")
        .highlight_min(subset=["TPR"], props="color:red;font-weight:bold")
        .set_properties(**{"text-align": "center"})
    )
    .format(
        {
            "selection_rate": "{:,.1%}".format,
            "TPR": "{:,.1%}".format,
            "FPR": "{:,.1%}".format,
        }
    )
    .set_properties(
        **{
            "font-size": "12pt",
        }
    )
)

dfi.export(to_blog, "fairness.png")

In [ ]:
to_blog

In [ ]:
colors = ["#EE3A64", "#028EA7"]
snsdata = metric_frame.by_group.copy()

fig, ax = plt.subplots(nrows=1, ncols=1)
sns.barplot(
    ax=ax,
    data=snsdata.reset_index(),
    x="etnisitet",
    y="selection_rate",
    hue="kjonn",
    palette=sns.color_palette(colors),
)

fig.savefig("dicriminate.png", dpi=100)

### Bedre sent enn aldri - vi utforsker treningsdata

In [ ]:
# pp.ProfileReport(df.loc[X_train.index, :])

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12, 4), sharex=False, sharey=True)
sns.boxplot(
    ax=ax[0],
    y=df.loc[X_train.index, "inntekt"],
    x=df.loc[X_train.index, "etnisitet"],
    palette=sns.color_palette(colors),
)
plt.ticklabel_format(style="plain", axis="y")
sns.boxplot(
    ax=ax[1],
    y=df.loc[X_train.index, "inntekt"],
    x=df.loc[X_train.index, "kjonn"],
    palette=sns.color_palette(colors),
)
plt.ticklabel_format(style="plain", axis="y")
fig.savefig("income.png", dpi=100)

#### Vi ser også på partielle avhengigheter i modellen - her med testdatasettet

In [ ]:
# Gjennomsnittlig partial dependency for alder og inntekt
fig, ax = plt.subplots(figsize=(12, 6))
ax.set_title("Neural net")
tree_disp = PartialDependenceDisplay.from_estimator(
    unmitigated_model, X_test, [0, 1, (0, 1)], ax=ax
)
fig.savefig("partialdeps.png", dpi=100)

## Biasdemping

In [ ]:
optimizer = ThresholdOptimizer(
    estimator=unmitigated_model,
    constraints="equalized_odds",
    predict_method="auto",
    objective="accuracy_score",
    grid_size=30000,
    flip=True,
)

In [ ]:
optimizer.fit(X_train, y_train, sensitive_features=A_train)

fig, ax = plt.subplots(nrows=1, ncols=1)
plot_threshold_optimizer(optimizer, ax=ax)


fig.savefig("biasdemping.png", dpi=100)

In [ ]:
# Checking model fairness using Fairlearn MetricFrame
metric_frame_optimized = MetricFrame(
    metrics={
        "accuracy": skm.accuracy_score,
        "precision": skm.precision_score,
        "auc": skm.roc_auc_score,
        "recall": skm.recall_score,
        "selection_rate": selection_rate,
        "count": count,
        "TPR": fairlearn.metrics.true_positive_rate,
        "FPR": fairlearn.metrics.false_positive_rate,
    },
    sensitive_features=A_test,
    y_true=y_test,
    y_pred=optimizer.predict(X_test, sensitive_features=A_test),
)

In [ ]:
metric_frame_optimized.overall

In [ ]:
metric_frame_optimized.by_group

In [ ]:
colors = ["#EE3A64", "#028EA7"]
snsdata = metric_frame_optimized.by_group.copy()

fig, ax = plt.subplots(nrows=1, ncols=1)
sns.barplot(
    ax=ax,
    data=snsdata.reset_index(),
    x="etnisitet",
    y="selection_rate",
    hue="kjonn",
    palette=sns.color_palette(colors),
)

fig.savefig("dicriminate.png", dpi=100)

In [ ]:
to_blog2 = (
    (
        metric_frame_optimized.by_group.drop(
            ["accuracy", "precision", "auc", "recall", "count"], axis=1
        )
        .copy()
        .style.highlight_max(subset=["FPR"], props="color:red;font-weight:bold")
        .highlight_min(subset=["FPR"], props="color:green;font-weight:bold")
        .highlight_max(subset=["selection_rate"], props="color:red;font-weight:bold")
        .highlight_min(subset=["selection_rate"], props="color:green;font-weight:bold")
        .highlight_max(subset=["TPR"], props="color:green;font-weight:bold")
        .highlight_min(subset=["TPR"], props="color:red;font-weight:bold")
        .set_properties(**{"text-align": "center"})
    )
    .format(
        {
            "selection_rate": "{:,.1%}".format,
            "TPR": "{:,.1%}".format,
            "FPR": "{:,.1%}".format,
        }
    )
    .set_properties(
        **{
            "font-size": "12pt",
        }
    )
)

dfi.export(to_blog2, "fairness_after_opimizer.png")